In [1]:
#importing libraries
import pandas as pd
import numpy as np

####  Read the books dataset and explore it


In [2]:
#reading in csv files
books=pd.read_csv("BX-Books.csv", encoding="utf-8") 
books.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,312956762,Die Hard: With a Vengeance - A Novel,D. Chiel,1995,St Martins Pr
1,034545006X,The Taking,J. D. Landis,2003,Ballantine Books
2,1579651372,Ruskin's Rose: A Venetian Love Story,Mimma Balia,2000,Artisan Publishers
3,416652700,The Pooh cook book: inspired by \Winnie-the-Po...,";Katie Stewart""",1971,Methuen
4,1853114103,Lord Have Mercy,"\\""Ron\""""",2001,Canterbury Press


In [3]:
books.tail()

,isbn,book_title,book_author,year_of_publication,publisher
271353,1573222623,Drinking Coffee Elsewhere (Today Show Book Clu...,ZZ Packer,2003,Riverhead Books
271354,1573222348,Drinking Coffee Elsewhere (Alex Awards (Awards)),ZZ Packer,2003,Riverhead Books
271355,1573223786,Drinking Coffee Elsewhere,Zz Packer,2004,Riverhead Books
271356,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing
271357,751352497,A+ Quiz Masters:01 Earth,NaN,1999,Dorling Kindersley


In [4]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271358 entries, 0 to 271357
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271358 non-null  object
 1   book_title           271358 non-null  object
 2   book_author          271356 non-null  object
 3   year_of_publication  271358 non-null  int64 
 4   publisher            271356 non-null  object
dtypes: int64(1), object(4)
memory usage: 10.4+ MB


#### Clean up NaN values

In [5]:
books[books.isna().any(axis=1)]# records containing null values

,isbn,book_title,book_author,year_of_publication,publisher
70531,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN
159822,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN
271356,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing
271357,751352497,A+ Quiz Masters:01 Earth,NaN,1999,Dorling Kindersley


In [6]:
#dropping records
books=books.dropna()

In [9]:
books.shape# 4 records have been dropped 

(271354, 5)